# Rules Extraction

# Statics

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.utils import Bunch
from sklearn.datasets._base import load_csv_data
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from iPRules.iPRules import iPRules
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from iPRules.iPRules import iPRules

import csv

## Load Dataset

In [2]:
# Load Dataset
target_true = '1'
target_false = '0'
test_size = 0.2
filename = 'clean_dataset'



data_file_name = f'../../../data/{filename}.csv'
pandas_dataset = pd.read_csv(data_file_name)

target_value_name = pandas_dataset.columns[-1]
feature_names = pandas_dataset.columns[0:-1]

pandas_dataset.head()


,EMERGENCY_DIA_SHORT_F05,EMERGENCY_DIA_SHORT_J12,EMERGENCY_DIA_SHORT_R06,EMERGENCY_DIA_SHORT_K52,EMERGENCY_DIA_SHORT_I10,EMERGENCY_DIA_SHORT_J22,EMERGENCY_DIA_SHORT_J84,EMERGENCY_DIA_SHORT_E87,EMERGENCY_DIA_SHORT_R09,EMERGENCY_DIA_SHORT_J98,...,ANTECEDENTS_PROC_BW03ZZZ,ANTECEDENTS_PROC_0TPBX0Z,ANTECEDENTS_PROC_F0796FZ,ANTECEDENTS_PROC_4A12X4Z,ANTECEDENTS_PROC_3E03329,SEXO,AGE_LOWER_40,AGE_40_60,AGE_HIGHER_60,RESULT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0,0,0,1,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0,0,1,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0,0,0,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1,0,0,1,1


# Divide dataset

In [3]:
X = pandas_dataset[feature_names]
y = pandas_dataset[target_value_name]

dataset = Bunch(
        data=X.to_numpy(),
        target=y.to_numpy(),
        target_names=target_value_name,
        feature_names=X.columns
)

## Define Random Forest

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, make_scorer, recall_score, accuracy_score

# Define scorer

## iPRules

In [5]:
n_splits = 10
chi_square_percent_point_function = 0.95
scale_feature_coefficient = 0.15
min_accuracy_coefficient = 0.9
min_number_class_per_node = 3
sorting_method="target_accuracy"
criterion="entropy"




cobertura_list = []
ensemble_accuracy_list = []
rules_accuracy_list = []

for train, test in KFold(n_splits = 10).split(X, y):

    ensemble = RandomForestClassifier(criterion=criterion)
    X_train = X.loc[train].to_numpy()
    y_train = y.loc[train].to_numpy()
    X_test = X.loc[test].to_numpy()
    y_test = y.loc[test].to_numpy()

    encoded_train_pandas_dataset = pd.DataFrame(data= np.c_[X_train, y_train], columns= list(dataset['feature_names']) + [target_value_name])
    encoded_test_pandas_dataset = pd.DataFrame(data= np.c_[X_test, y_test], columns= list(dataset['feature_names']) + [target_value_name])

    rules = iPRules(
                    feature_names=dataset.feature_names,
                    target_value_name = dataset.target_names,
                    chi_square_percent_point_function = chi_square_percent_point_function,
                    scale_feature_coefficient = scale_feature_coefficient,
                    min_accuracy_coefficient = min_accuracy_coefficient,
                    min_number_class_per_node = min_number_class_per_node
                )
    # Fit model
    ensemble.fit(X_train, y_train)
    rules.fit(encoded_train_pandas_dataset, ensemble.feature_importances_)

    # Predict
    y_pred_test_ensemble = ensemble.predict(X_test)
    y_pred_test_rules = rules.predict(X_test, sorting_method=sorting_method)

    # Reduce Dataset
    np_array_rules = np.array(y_pred_test_rules)
    #not_filter_indices = np.where(np.logical_and(np_array_rules != 0, np_array_rules!=1))[0]
    filter_indices = np.where(np_array_rules != None)[0]
    np_filterred_y_test = np.array(y_test)[filter_indices]
    np_filterred_y_pred_test_ensemble = np.array(y_pred_test_ensemble)[filter_indices]
    np_filterred_y_pred_test_rules = np.array(y_pred_test_rules)[filter_indices].astype('int64')

    # SCORERS
    cobertura = len(np_filterred_y_pred_test_rules)/len(y_test)
    cobertura_list.append(cobertura)

    ensemble_accuracy = metrics.accuracy_score(np_filterred_y_test, np_filterred_y_pred_test_ensemble)
    ensemble_accuracy_list.append(ensemble_accuracy)

    rules_accuracy = metrics.accuracy_score(np_filterred_y_test, np_filterred_y_pred_test_rules)
    rules_accuracy_list.append(rules_accuracy)


np_cobertura = np.array(cobertura_list)
np_ensemble_accuracy = np.array(ensemble_accuracy_list)
np_rules_accuracy = np.array(rules_accuracy_list)

print("%0.2f np_cobertura with a standard deviation of %0.2f" % (np_cobertura.mean(), np_cobertura.std()))
print("%0.2f np_ensemble_accuracy with a standard deviation of %0.2f" % (np_ensemble_accuracy.mean(), np_ensemble_accuracy.std()))
print("%0.2f np_rules_accuracy with a standard deviation of %0.2f" % (np_rules_accuracy.mean(), np_rules_accuracy.std()))

[82.28571428571428, 84.57142857142857, 80.57142857142857, 84.0, 57.47126436781609, 87.35632183908046, 59.77011494252874, 81.60919540229885, 67.24137931034483, 54.59770114942529]
[0.9166666666666666, 0.8986486486486487, 0.8936170212765957, 0.9319727891156463, 0.95, 0.8881578947368421, 0.9711538461538461, 0.8591549295774648, 0.9658119658119658, 0.9894736842105263]
[0.9097222222222222, 0.9054054054054054, 0.8936170212765957, 0.891156462585034, 0.95, 0.8881578947368421, 0.9615384615384616, 0.852112676056338, 0.9743589743589743, 0.9894736842105263]
73.95 np_cobertura with a standard deviation of 12.07
0.93 np_ensemble_accuracy with a standard deviation of 0.04
0.92 np_rules_accuracy with a standard deviation of 0.04
